In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
from decouple import config

In [30]:
from repomanager.github import  GithubClient

In [31]:
client = GithubClient(config("GITHUB_TOKEN"))

In [32]:
print(client)

GithubClient(login=giocaizzi, repos=18)


In [33]:
client.user

AuthenticatedUser(login="giocaizzi")

In [62]:
client.user.__dict__

{'_requester': <github.Requester.Requester at 0x10e782e90>,
 '_avatar_url': <github.GithubObject._ValuedAttribute at 0x10e6d3b90>,
 '_bio': <github.GithubObject._ValuedAttribute at 0x10e6b5750>,
 '_blog': <github.GithubObject._ValuedAttribute at 0x10e6b5210>,
 '_collaborators': NotSet,
 '_company': <github.GithubObject._ValuedAttribute at 0x10e6b5050>,
 '_created_at': <github.GithubObject._ValuedAttribute at 0x10e6b4fd0>,
 '_disk_usage': NotSet,
 '_email': <github.GithubObject._ValuedAttribute at 0x10e6b6450>,
 '_events_url': <github.GithubObject._ValuedAttribute at 0x10e6b4490>,
 '_followers': <github.GithubObject._ValuedAttribute at 0x10e6b6390>,
 '_followers_url': <github.GithubObject._ValuedAttribute at 0x10e6b41d0>,
 '_following': <github.GithubObject._ValuedAttribute at 0x10e6b6290>,
 '_following_url': <github.GithubObject._ValuedAttribute at 0x10e6b4c10>,
 '_gists_url': <github.GithubObject._ValuedAttribute at 0x10e6b5010>,
 '_gravatar_id': <github.GithubObject._ValuedAttribute 

In [34]:
client.login

'giocaizzi'

In [35]:
client.name

'Giorgio Caizzi'

## Repository info

In [36]:
client.repos

[Repository(name=aecom_3dmodels),
 Repository(name=cookiecutter-python-ghpages),
 Repository(name=cover-letter),
 Repository(name=CV),
 Repository(name=giocaizzi),
 Repository(name=gsheet-monthly-hustle),
 Repository(name=magicguide),
 Repository(name=mplStrater),
 Repository(name=py3dinterpolations),
 Repository(name=pyarcgis),
 Repository(name=pyfolio),
 Repository(name=pysurfline),
 Repository(name=repomanager),
 Repository(name=spatial-interpolation),
 Repository(name=spotify_to_tidal),
 Repository(name=surfingcrypto),
 Repository(name=TelegramBotNotifications),
 Repository(name=wiki)]

In [37]:
client.repos[0].url

'https://github.com/giocaizzi/aecom_3dmodels'

In [57]:
client.repos[0].issues.totalCount

0

In [1190]:
repo_contents = [
    {
        "Name": repo.name,
        "Language": repo.language,
        "Description": repo.description,
        "Url": repo.html_url,
        "Stars": repo.stargazers_count,
        "Issues": {
            "Open": sum([issue.state == "open" for issue in repo.get_issues()]),
            "Total": repo.get_issues().totalCount,
        },
        "Pull Requests": {
            "Open": sum([issue.state == "open" for issue in repo.get_pulls()]),
            "Total": repo.get_pulls().totalCount,
        },
        "Branches": {
            "Default": repo.default_branch,
            "Branches": [branch.name for branch in repo.get_branches()],
            "Total": repo.get_branches().totalCount,
        },
        "Forks": {
            "Forks": [fork.full_name for fork in repo.get_forks()],
            "Total": repo.forks_count,
        },
        "Pages": repo.has_pages,
        "Collaborators": {
            "Collaborators": [
                collaborator.login for collaborator in repo.get_collaborators()
            ],
            "Total": repo.get_collaborators().totalCount,
        },
        "Workflows": {
            "Workflows": [workflow.name for workflow in repo.get_workflows()],
            "Total": repo.get_workflows().totalCount,
        },
    }
    for repo in repos
]

In [1191]:
# Set up the environment
env = Environment(loader=FileSystemLoader('.'),extensions=['jinja2.ext.loopcontrols'])
template = env.get_template('template.html')

In [1192]:
# custom filter jinja2

def is_true(value):
    return isinstance(value, bool) and value

def is_false(value):
    return isinstance(value, bool) and not value

# Add the filter to the Jinja2 environment
env.filters['is_true'] = is_true
env.filters['is_false'] = is_false

In [1193]:
# html = template.render(repos=[vars(repo) for repo in repos])
html = template.render(repos=repo_contents)

In [1194]:
HTML(html)

In [1195]:
with open('index.html', 'w') as file:
    file.write(html)